<a href="https://colab.research.google.com/github/cesaenv/buscaminas/blob/main/3_1_An%C3%A1lisisEstad%C3%ADsticoComparaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 3.1. Análisis estadístico comparativo entre modelos


### 1. Dataset

Antes de comenzar vamos a cargar las librerías que serán necesarias. Con respecto al notebook 1, habría que cargar numpy y pandas.

In [3]:
import numpy as np
import pandas as pd


Seguiremos con el dataset train.csv. Un dataset que intenta predecir la actividad que estaba realizando un paciente a partir de distintos datos, los correspondientes a las columnas del dataset.

In [4]:
df = pd.read_csv('train.csv')

Nuestro dataset constituye a una recopilación de datos por cada paciente, por lo que vamos a describir cada una de las columnas (o datos). A continuación se muestra la descripción de las distintas características de este dataset.


|  Name  | Description | Value Type |
|---         |---       |---    
| **SL** | Nivel de azúcar | Numerical |
| **EEG** | Ratio monitor EEG | Numerical |
| **BP** | Presión en sangre | Numerical |
| **HR** | Ratio de latidos | Numerical |
| **CIRCULATION** | Circulación de sangre | Numerical |

Por otro lado, *ACTIVITY* es la predicción de la actividad que estaba realizando un paciente a partir de los anteriores datos. Es el que tenemos que predecir. Posibles valores:

|  Valor  | Description
|---         |---         
| **0** | De pie |
| **1** | Caminando |
| **2** | Sentado |
| **3** | Cayendo |
| **4** | Con calambres |
| **5** | Corriendo |

Para evaluar todos los conjuntos de la misma manera, tendré que evaluarlos con la misma partición del dataset. Por ello, realizamos ya la partición.

Primero cargamos la librería:

In [6]:
from sklearn.model_selection import train_test_split

Luego tenemos que dividir a nuestro conjunto de entrenamiento en los propios datos y las correspondientes etiquetas:

In [17]:
X =  df.values[:,:-1]
Y =  df.values[:,-1]

Y luego hacemos la partición, asignando al conjunto test un 25%

In [18]:
(trainData, testData, trainLabels, testLabels) = train_test_split(X,Y,test_size=0.25, random_state=42)

Además, importaremos aquí directamente las librerías que son necesarias para cada uno de los algoritmos de clasificación:

In [12]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
#Árboles de decisión
from sklearn.tree import DecisionTreeClassifier
#Regresión logística
from sklearn.linear_model import LogisticRegression
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
#Redes neuronales
from sklearn.linear_model import Perceptron #para el tipo de red neuronal Perceptron (el más sencillo)
from sklearn.neural_network import MLPClassifier #para el tipo de red neuronal Perceptron pero multicapa
#RandomSearch
from scipy.stats import randint as sp_randint

Además, como en la realización del notebook me molestaban los warnings he decidido ignorarlos:

In [5]:
import warnings

# Ignorar todos los warnings
warnings.filterwarnings("ignore")

Lo último que vamos a ver en esta parte es cómo realizar un estudio estadístico basado en lo que vimos en clase, para ello utilizaremos la función `compare_methods` disponible en el módulo StatisticalAnalysis. Comienzo instalando este módulo usando `pip`.

In [1]:
!pip install StatisticalAnalysis

  Preparing metadata (setup.py) ... done
  Created wheel for StatisticalAnalysis: filename=StatisticalAnalysis-0.0.5-py2.py3-none-any.whl size=13098 sha256=2cc28330b442b2d678216f905454b12b6a546ff4ef86425b5ee3be5852f877c9
  Stored in directory: /root/.cache/pip/wheels/8c/db/95/26b1f6f0da09912e26583c42371b4ac0d4fd1a8348a8636b6b
Successfully built StatisticalAnalysis


## Estudio Estadístico

In [8]:
from StatisticalAnalysis import compare_methods

Para utilizar este método debemos comenzar indicando los algoritmos a utilizar y las distribuciones de los hiperparámetros a optimizar. Vamos a utilizar los 5 modelos de la parte de Aprendizaje Supervisado vistos en 3. Comparativa de algoritmos: árboles de decisión, SVMs, KNN, Regresión logística y red neuronal, así que definimos estos modelos y los parámetros a optimizar.

In [13]:
# Árbol de decisión
clfTree = DecisionTreeClassifier(random_state=84)
param_distTree = {"min_samples_split": sp_randint(3, 30)}
# SVM
clfSVC = SVC(random_state=84)
param_distSVC = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],'kernel': ['rbf'], 'class_weight':['balanced', None]}
# KNN
clfKNN = KNeighborsClassifier()
param_distKNN = {'n_neighbors':sp_randint(3, 30)}
# Regresión logística
clfLR = LogisticRegression(random_state=84)
param_distLR = {'C': [0.1,0.5,1, 10, 100, 1000]}
# Red neuronal
clfMLP = MLPClassifier(random_state=84)
param_distMLP = {'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'],
                 'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'learning_rate': ['constant', 'invscaling', 'adaptive'],
                 'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)],
                 'momentum': [0.9, 0.95, 0.99]}

Creamos tres listas que contendrán respectivamente:

In [14]:
#Los algoritmos
listAlgorithms = [clfTree,clfSVC,clfKNN,clfLR,clfMLP]
#Los parámetros a optimizar
listParams = [param_distTree,param_distSVC,param_distKNN,param_distLR,param_distMLP]
#Los nombres de los algoritmos
listNames = ["Arbol", "SVM", "KNN", "LR", "MLP"]

Invocamos a la función `compare_methods` pasándole el dataset completo, el conjunto de etiquetas completo, las tres listas definidas anteriormente, y la métrica. Los valores válidos
para la métrica son accuracy, precision, recall, f1 o auroc. Al invocar a la función anterior se producirá un informe de resultados.

### Estudio estadístico con métrica accuracy

In [ ]:
compare_methods(X,Y,listAlgorithms,listParams,listNames,metric='accuracy')
#función dada gracias a:  from StatisticalAnalysis import compare_methods

In the multiclass problem, only accuracy can be used as metric
In the multiclass problem, only accuracy can be used as metric
